# What is Micrograd?

"**Micrograd**" is small autograd engine. The word **Autograd** is short for "**Automatic Gradient**". Autograd is also the name of PyTorch's automatic differentiation system.

What "Micrograd" does, is that it implements the "Backpropagation" algorithm. "Backprop" helps us **efficiently compute the gradient of a loss/error function with respect to the weights of a given neural network**.

With the gradient, we can iterativelly tune/tweak the weights of a network to minimize our loss function.

# Why Am I learning this? 💡🧠

I can right away jump into building neural nets
I decided to learn how to build because:

1. I want deep understanding, and practice patience. As a newcomer, it is easy to fall into

2. Backpropagation is at the CORE of any modern deep learning frameworks.